In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

folder_format = "lukas"
recording = "/media/kuka/Seagate Expansion Drive/kuka_recordings/flow/control_test"

episode_num = 1
state_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
flow_recording_fn = "{}/episode_{}.npz".format(recording, episode_num)
mask_recording_fn = "{}/episode_{}_mask.npz".format(recording, episode_num)
keep_fn = "{}/episode_{}_keep.npz".format(recording, episode_num)

state_recording = np.load(state_recording_fn)["robot_state_full"]
actions = np.load(state_recording_fn)["actions"]
ee_positions = state_recording[:,:3]

flow_recording = np.load(flow_recording_fn)["rgb_unscaled"]
num_frames = flow_recording.shape[0]-1
masks = np.load(mask_recording_fn)["mask"]
np.savez(keep_fn, keep=np.ones(num_frames,dtype=bool))

print(mask_recording_fn)
print(state_recording.shape)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = flow_recording[i].copy()
    mask = masks[i]
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)

In [ ]:
from gym_grasping.flow_control.servoing_module import ServoingModule
servo_module = ServoingModule(recording, episode_num=episode_num, plot=True)

In [ ]:
start_frame = 140
target_frame = 145

servo_module.set_base_frame(target_frame)

In [ ]:
image_start = flow_recording[start_frame].copy()
image_start[np.logical_not(masks[start_frame])] = 255

image_target = flow_recording[target_frame].copy()
image_target[np.logical_not(masks[target_frame])] = 255

# state has the structure: x,y,z,rot_z,rot_y,rot_x,joint[0-6], desired tcp pos x,y,z, rot_z, force x,y,z, torque x,y,z
print(state_recording[start_frame])
print(state_recording[target_frame])



fig, ax = plt.subplots(1,1)
plt_handle = ax.imshow(image_start)
ax.set_axis_off()

def update(i):
    if i == 0:
        plt_handle.set_data(image_start)
    if i == 1:
        plt_handle.set_data(image_target)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=1, step=1,value=0,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)
plt.show()

In [ ]:
obs_image = flow_recording[start_frame]
obs_ee_pos = ee_positions[start_frame]
action,_ = servo_module.step(obs_image, obs_ee_pos)

print(actions[start_frame])
print(action)

In [ ]:
 def step(self, state, ee_pos):
        # 1. compute flow
        # 2. compute transformation
        # 3. transformation to control

        assert(state.shape == self.base_image.shape)

        # Control computation
        if self.forward_flow:
            flow = self.flow_module.step(state, self.base_image)
        else:
            flow = self.flow_module.step(self.base_image, state)


        # Do the masked reference computation here.
        # because I need to use the FG mask
        assert(self.forward_flow == False)

        # x = np.linspace(-1, 1, self.size[0])
        # y = np.linspace(-1, 1, self.size[1])
        # xv, yv = np.meshgrid(x, y)
        #
        # field =  np.stack((yv, -xv), axis=2)
        # field[:,:,0] *= (84-1)/2*self.size[0]/256
        # field[:,:,1] *= (84-1)/2*self.size[1]/256
        # points = field

        # observations = points + flow
        #select foreground
        # points = points[self.base_mask]

        # array shape: height x width
        points = np.array(np.where(self.base_mask)).T.astype('float')
        # print(self.base_mask.shape)
        # observations = observations[self.base_mask]
        masked_flow = flow[self.base_mask]
        observations = points + masked_flow[:,::-1]

        observations -= np.array([240, 320])
        points -= np.array([240, 320])

        scaling_factor = (84-1)/2*1/256
        observations *= scaling_factor
        points *= scaling_factor
        #
        # fig = plt.figure()
        # ax = fig.add_subplot(111)  # , projection='3d')
        # ax.scatter(points[:, 1], points[:, 0])
        # ax.scatter(observations[:, 1], observations[:, 0])
        #
        # ax.scatter([0], [0])
        # ax.set_xlim([int(-320 * scaling_factor),int(320 * scaling_factor)])
        # ax.set_ylim([int(240* scaling_factor),int(-240* scaling_factor)])
        # ax.set_aspect('equal')
        # plt.show()
        # print(points)
        # print(points.mean(axis=0))


        points = np.pad(points, ((0, 0), (0, 2)), mode="constant")
        observations = np.pad(observations, ((0, 0), (0, 2)), mode="constant")
        guess = solve_transform(points, observations)
        r = R.from_dcm(guess[:3,:3])
        xyz = r.as_euler('xyz')
        rot_z = xyz[2]
        # magical gain values for control, these could come from calibration
        mean_flow = guess[0,3]/23, guess[1,3]/23
        mean_rot = -7*rot_z

        if not self.forward_flow:
            # then reverse direction of actions
            mean_flow = np.array(mean_flow) * -1
            mean_rot = mean_rot * -1
        # Outputs of this block: used at beginning of loop
        #   1. mean_flow
        #   2. mean_rot

        pos_diff = self.base_pos - ee_pos
        loss_z = np.abs(pos_diff[2])*10
        loss_pos = np.linalg.norm(mean_flow)
        loss_rot = np.abs(mean_rot) / 1.5
        loss = loss_pos + loss_rot + loss_z

        z = pos_diff[2] * 10 * 3
        action = [mean_flow[0], -mean_flow[1], z, mean_rot, self.grip_state]


        # plotting code
        if self.view_plots:
            # show flow
            flow_img = self.flow_module.computeImg(flow, dynamic_range=False)
            # show segmentatione edge
            if self.counter % 5 == 0:
                edge  = np.gradient(self.base_mask.astype(float))
                edge = (np.abs(edge[0])+ np.abs(edge[1])) > 0
                flow_img[edge] = (255, 0, 0)

            print("loss =", loss, action)
            # show loss, frame number
            self.view_plots.step(loss, self.base_frame, self.base_pos[2], ee_pos[2])
            self.view_plots.low_1_h.set_data(state)
            self.view_plots.low_2_h.set_data(self.base_image)
            self.view_plots.low_3_h.set_data(flow_img)
            plot_fn = f'./video/{self.counter:03}.png'
            #plt.savefig(plot_fn, bbox_inches=0)

            
        self.counter += 1
        return action, self.mode